In [2]:
"""
Created on Sun Nov 24 11:23:09 2019

@author: marco
"""

"""
this is the code you need to run to import the data. 
You just have to change line 40 putting the correct path.
"""

import numpy as np

def load_mnist(path, kind='train'):
    import os
    import gzip
    import numpy as np

    """Load MNIST data from `path`"""
    labels_path = os.path.join(path,
                               '%s-labels-idx1-ubyte.gz'
                               % kind)
    images_path = os.path.join(path,
                               '%s-images-idx3-ubyte.gz'
                               % kind)

    with gzip.open(labels_path, 'rb') as lbpath:
        labels = np.frombuffer(lbpath.read(), dtype=np.uint8,
                               offset=8)

    with gzip.open(images_path, 'rb') as imgpath:
        images = np.frombuffer(imgpath.read(), dtype=np.uint8,
                               offset=16).reshape(len(labels), 784)

    return images, labels



X_all_labels, y_all_labels = load_mnist('C:/Users/RE-Giorgio/Documents/OptimusPrime/Data', kind='train')


"""
We are only interested in the items with label 2, 4 and 6.
Only a subset of 1000 samples per class will be used.
"""
indexLabel2 = np.where((y_all_labels==2))
xLabel2 =  X_all_labels[indexLabel2][:1000,:].astype('float64') 
yLabel2 = y_all_labels[indexLabel2][:1000].astype('float64') 

indexLabel4 = np.where((y_all_labels==4))
xLabel4 =  X_all_labels[indexLabel4][:1000,:].astype('float64') 
yLabel4 = y_all_labels[indexLabel4][:1000].astype('float64') 

indexLabel6 = np.where((y_all_labels==6))
xLabel6 =  X_all_labels[indexLabel6][:1000,:].astype('float64') 
yLabel6 = y_all_labels[indexLabel6][:1000].astype('float64') 


"""
To train a SVM, you have to convert the labels of the two classes of interest into '+1' and '-1'.
"""
yLabel2[:] = +1
yLabel4[:] = -1

X = np.concatenate([xLabel2, xLabel4])
y = np.concatenate([yLabel2, yLabel4])


In [3]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X=scaler.fit_transform(X)

In [ ]:
class Svm:
    
    def __init__(self, X, y, gamma):
        
        self.X = X.T
        self.y = y
        self.alpha = np.random.normal(0,1,(self.X.shape[1],1))
        self.b = np.random.normal(0,1,(self.X.shape[1],1))
        self.gamma = gamma
        
    def predict(self,X, y):
        
        z = self.alpha*y*self.kernel_poly(X) + self.b
        a = np.sign(z)    
        return a
        
    def kernel_gauss(self, X):
        
        radius = np.split(X, X.shape[0]) - X
        return np.exp(-self.gamma * (np.linalg.norm(radius, axis = 0)))
    
    def kernel_poly(self,x):
        
        return (np.split(X, X.shape[0]) @ np.stack([X.T]*X.shape[0]) + 1)**self.gamma
                     
svm = Svm(X, y, gamma = 6)
svm.predict(X,y)

In [51]:
X = np.arange(30).reshape(10,3)

np.split(X, X.shape[0]) @ np.stack([X.T]*X.shape[0])

array([[[   5,   14,   23,   32,   41,   50,   59,   68,   77,   86]],

       [[  14,   50,   86,  122,  158,  194,  230,  266,  302,  338]],

       [[  23,   86,  149,  212,  275,  338,  401,  464,  527,  590]],

       [[  32,  122,  212,  302,  392,  482,  572,  662,  752,  842]],

       [[  41,  158,  275,  392,  509,  626,  743,  860,  977, 1094]],

       [[  50,  194,  338,  482,  626,  770,  914, 1058, 1202, 1346]],

       [[  59,  230,  401,  572,  743,  914, 1085, 1256, 1427, 1598]],

       [[  68,  266,  464,  662,  860, 1058, 1256, 1454, 1652, 1850]],

       [[  77,  302,  527,  752,  977, 1202, 1427, 1652, 1877, 2102]],

       [[  86,  338,  590,  842, 1094, 1346, 1598, 1850, 2102, 2354]]])